In [3]:
# l'objectif est de récupérer un maximum d'annonces immobilières sur un site "scrappable".
#      Ce n'est pas le cas pour de nombreux sites comme le bon coin, ou seloger.com protégé par une captcha.
#      En cherchant un peu j'ai trouvé un site qui ne présente pas ce genre de protection : https://www.superimmo.com/
#      D'autant plus que le système de pagination (via l'url) facilite la récupération des données.
# Contexte :
#      ici on cherchera à établir des tendances du prix de l'immobilier parisien.
#      une fois qu'on rempli le formulaire les éléments de recherche se retrouve dans l'url
#      Ainsi on peut accéder aux annonces en saisissant directement : 
#           https://www.superimmo.com/achat/ile-de-france/paris?option%5B%5D=old_build&option%5B%5D=new_build
#      On observe que par la suite c'est l'élément "p/3" (ou 3 désigne la page 3) de l'url :
#           https://www.superimmo.com/achat/ile-de-france/paris/p/3?option%5B%5D=old_build&option%5B%5D=new_build
#      qui permet d'obtenir tous les biens de ladite page.
#      par ailleurs on observe qu'il s'agit de la page 3/791.
#      Il suffit d'itérer l'argument p/n jusqu'à 791 pour obtenir toutes les annonces du site pour cette recherche.
#      cependant on a pas le details de chaque annonces
# Etapes : 
#      1/ Dans un premier temps en passage sur chaque page de 1 à 791 
#           on cherchera à obtenir quelques premières informations
#           mais surtout à obtenir le liens vers le détails de chaque annonce.
#      2/ Ensuite pour chaque lien on tachera de récuperer un maximum d'informations pertinentes
# Protocole : 
#      Ici on utilisera selenium notamment parce qu'il faut cliquer sur un bouton pour les cookies 
#      et aussi parce qu'il permet de suivre visuellement son traitement.
#      On construira un alimentera un dictionnaire avec les éléments recueillits pour chaque annonces 
#      que l'on transformera en dataframe pandas
#      puis l'on enregistrera dans un fichier csv
#      par la suite on extraira et formatera les données qui nous intéressent pour analyse. 

In [4]:
# requirements 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [5]:
# acces au site 
driver = webdriver.Firefox()
driver.get("https://www.superimmo.com/achat/ile-de-france/paris?option%5B%5D=old_build&option%5B%5D=new_build")

time.sleep(3) # le temps que le popup pour les cookies apparaisse

# on clique sur le bouton
NEXT_BUTTON_XPATH = '//button[@id="tarteaucitronPersonalize2"]'
button = driver.find_element_by_xpath(NEXT_BUTTON_XPATH)
button.click()
# on est prêt à scrapper

In [6]:
# recensement des annonces

# on commence par désigné les pages de liste d'annonces qui nous intéressent
pages = []
# fait par paquet de 200 histoire de ne pas tout perdre si ça plante en cours de route ...
init=0  # il suffit d'ajouter 200 à chaque itération
for n in range(1):
    pages.append('https://www.superimmo.com/achat/ile-de-france/paris/paris/p/'+str(init+n)+'?option%5B%5D=old_build&option%5B%5D=new_build',)

# on désigne les élements de la liste d'annonces qui nous intéresse
synthese = {
    "prix":[],
    "titre":[],
    "link":[],
    "content":[]
};

for page in pages :
    driver.get(page) # on lit la page
    blocks = driver.find_elements_by_tag_name('article') # qui désigne le resumé d'une annonces
    for block in blocks :
        if block.find_elements_by_css_selector('.prix'): 
            # print(block.find_element_by_class_name('prix').text)
            synthese['prix'].append(block.find_element_by_class_name('prix').text)
        else : 
            synthese['prix'].append(0) # on saisit une valeur même si rien n'est trouvé
            
        if block.find_elements_by_css_selector('.titre'): 
            # print(block.find_element_by_class_name('titre').text)
            synthese['titre'].append(block.find_element_by_class_name('titre').text)
        else:
            synthese['titre'].append(0)# même chose ici
            
        if block.find_element_by_tag_name('a'): 
            # print(block.find_element_by_tag_name('a').get_attribute("href"))
            synthese['link'].append(block.find_element_by_tag_name('a').get_attribute("href"))
        else : 
            synthese['link'].append(0)
            
        if block.find_elements_by_css_selector('.media-body'):
            # on concatène les autres éléments 
            contents = block.find_elements_by_css_selector('.media-body')
            synthese['content'].append(''.join(e.text for e in contents))
        else : 
            synthese['content'].append(0)

# print(synthese)
        
# dictionnaire to dataframe
synthese = pd.DataFrame(synthese)
print(synthese)

# sauvegarde csv des datas : 
# synthese.to_csv('superimmo_20211110_list_'+str(init)+'_'+str(init+200)+'.csv')

           prix                 titre  \
0   5 890 000 €    Appartement 265 m²   
1     674 000 €     Appartement 55 m²   
2     319 000 €     Appartement 23 m²   
3   2 190 000 €    Appartement 175 m²   
4     749 000 €     Appartement 50 m²   
5     750 000 €     Appartement 58 m²   
6     360 000 €  Appartement 39,41 m²   
7     449 000 €     Appartement 54 m²   
8     577 200 €     Appartement 57 m²   
9     747 000 €     Appartement 85 m²   
10            0                     0   
11            0                     0   

                                                 link  \
0   https://www.superimmo.com/annonces/achat-appar...   
1   https://www.superimmo.com/annonces/achat-appar...   
2   https://www.superimmo.com/annonces/achat-appar...   
3   https://www.superimmo.com/annonces/achat-appar...   
4   https://www.superimmo.com/annonces/achat-appar...   
5   https://www.superimmo.com/annonces/achat-appar...   
6   https://www.superimmo.com/annonces/achat-appar...   
7   https:

In [7]:
# # concatenation des resultats précédent trouvés
# import glob
# filenames = glob.glob("superimmo_20211008_69_*.csv")
# df = pd.DataFrame()
# # lecture des fichiers 
# for f in filenames : 
#     df = df.append( pd.read_csv(f))
    
# print(df)
# df.to_csv("superimmo_20211008_69_list.csv")

      Unnamed: 0       prix               titre  \
0              0  310 000 €   Appartement 86 m²   
1              1  549 000 €   Appartement 92 m²   
2              2  122 000 €       Maison 117 m²   
3              3  380 000 €   Appartement 53 m²   
4              4  199 000 €   Appartement 46 m²   
...          ...        ...                 ...   
2395        2395  620 000 €  Appartement 116 m²   
2396        2396  399 000 €       Maison 170 m²   
2397        2397  256 500 €   Appartement 41 m²   
2398        2398          0                   0   
2399        2399          0                   0   

                                                   link  \
0     https://www.superimmo.com/annonces/achat-appar...   
1     https://www.superimmo.com/annonces/achat-appar...   
2     https://www.superimmo.com/annonces/achat-maiso...   
3     https://www.superimmo.com/annonces/achat-appar...   
4     https://www.superimmo.com/annonces/achat-appar...   
...                              

In [7]:
# analyse des premiers resultats trouvés

# df = pd.read_csv('superimmo_20211110_list_0_200.csv')
df = synthese
print(df)
df.info()

# on observe que des lignes sont quasi-vides 
# ce sont des pubs qui se glisse au milieu des annonces (les liens récupéré ne ressemble pas à ceux d'une annonce)
# on exclura ces données en observant que dans ces cas là il n'y a pas de prix.
df = df[(df['prix']!='0')]

           prix                 titre  \
0   5 890 000 €    Appartement 265 m²   
1     674 000 €     Appartement 55 m²   
2     319 000 €     Appartement 23 m²   
3   2 190 000 €    Appartement 175 m²   
4     749 000 €     Appartement 50 m²   
5     750 000 €     Appartement 58 m²   
6     360 000 €  Appartement 39,41 m²   
7     449 000 €     Appartement 54 m²   
8     577 200 €     Appartement 57 m²   
9     747 000 €     Appartement 85 m²   
10            0                     0   
11            0                     0   

                                                 link  \
0   https://www.superimmo.com/annonces/achat-appar...   
1   https://www.superimmo.com/annonces/achat-appar...   
2   https://www.superimmo.com/annonces/achat-appar...   
3   https://www.superimmo.com/annonces/achat-appar...   
4   https://www.superimmo.com/annonces/achat-appar...   
5   https://www.superimmo.com/annonces/achat-appar...   
6   https://www.superimmo.com/annonces/achat-appar...   
7   https:

In [8]:
import re
import requests
import os



# ensuite on va chercher le maximum d'information que l'on peut récupérer pour chaque annonces

# on fait ça aussi par étape histoire de ne pas trop rager si ça plante en plein milieu
init = 1
step = 10

# recense les failled scraping si l'annonce n'existe plus par exemple : 
#      visiblement superimmo est un site plutot actif :  
#          une annonce affiché le matin n'est déjà plus en ligne l'après midi
# on veut empêcher qui le programme s'arrête s'il ne trouve pas la page demandé
failled=[]


# inventaire des informations qu'on va recueillir sur chaque annonce
synthese = {
    "link":[],
    "ref":[],
    "ref_si":[],
    "location":[],
    "agence":[],
    "description":[],
    "details":[],
    "titre":[]
};

for annonce in df[(df.index>=init)&(df.index<init+step)].link:
    try : 
        # lecture de l'annonce
        driver.get(annonce)

        # référence de l'annonce
        if driver.find_element_by_class_name('listing-references'):
            ref = driver.find_element_by_class_name('listing-references').text
            
            result = re.search(r".* Réf Superimmo : (\w*)", ref)
            ref_si= result.group(1)
            synthese['ref_si'].append(ref_si)
            
            # sauvegarde des photos de l'annonce: 
            images = driver.find_elements_by_class_name('fancybox')
            print(ref_si)
            print(len(images))
            if not os.path.exists('Photos_annonces/'+ref_si) :
                os.makedirs('Photos_annonces/'+ref_si)
            for n in range(len(images)) : 
                with open('Photos_annonces/'+ref_si+'/'+ref_si+'_'+str(n)+'.jpg', 'wb') as file:
                    r = requests.get(images[n].get_attribute("href"), allow_redirects=True)
                    file.write(r.content)
                
            # qui permet d'éviter des erreurs de format
            synthese['ref'].append(ref)
            synthese['link'].append(annonce) 
            
        else : 
            # on saisit un valeur même si rien n'est trouvé 
            # notamment pour respecter le format et éviter les erreurs de lecture
            synthese['ref'].append('')
        
        # description de l'annonces
        if driver.find_element_by_class_name('description'): 
            synthese['description'].append(driver.find_element_by_class_name('description').text)
        else : 
            synthese['description'].append('')

        # données du biens
        # qui sont toutes les informations concernant l'annonce.
        # visiblement le format est assez libre et variables suivant les annonces
        # cependant toutes comporte au moins le "Prix de vente : ..."
        blocks = driver.find_elements_by_tag_name('td')
        synthese['details'].append('|'.join([b.text for b in blocks]))

        # exemple Paris 5ème (75005)
        if driver.find_element_by_class_name('location'):
            synthese['location'].append(driver.find_element_by_class_name('location').text)
        else : 
            synthese['location'].append('')
        
        # agence en charge de vendre le bien
        if driver.find_elements_by_css_selector('.block_agence'):
            agence = driver.find_element_by_class_name('block_agence')
            if agence.find_elements_by_tag_name('b'):
                synthese['agence'].append(agence.text)
            else : 
                synthese['agence'].append('')
        else : 
            synthese['agence'].append('')
        
        # contenu du header 
        if driver.find_element_by_tag_name('h1'):
            synthese['titre'].append(driver.find_element_by_tag_name('h1').text)
        else : 
            synthese['titre'].append('')
            
    except :
        failled.append(annonce)
        continue


#### debug: permet de voir quels saisie données a échoué et rend la transformation en dataframe impossible 
##for k in synthese.keys():
##    print(k)
##    print(len(synthese[k]))

# transformation en dataframme
#print(synthese)
for k,v in synthese.items():
    print(k)
    print(len(v))
synthese = pd.DataFrame(synthese)
#print(synthese)

# print(synthese.header)
# enregistrement des resultats
print(synthese.to_csv('superimmo_20211110_75_annonces_'+str(init)+'_'+str(init+step)+'.csv'))

# affichage des echecs de parsing 
print('failled : ')
print(failled)

ANVXLWRA1
11
ANVXMXWUB
17
ANVXMNF3T
1
ANVXMGR9H
39
ANVXLYA7N
15
ANVXMM8Q3
13
ANVXM3UV6
17
ANVXLAR02
27
link
8
ref
8
ref_si
8
location
8
agence
8
description
8
details
8
titre
8
None
failled : 
['https://www.superimmo.com/annonces/achat-appartement-39m-paris-19eme-75019-xmk0xf', 'https://www.superimmo.com/le-mag/actualite-comment-faire-baisser-les-frais-de-notaire']


In [8]:
# # concatenation des resultats précédent trouvés
# import glob
# filenames = glob.glob("superimmo_20211008_13_annonces_*.csv")
# df = pd.DataFrame()
# lecture des fichiers 
# for f in filenames : 
#     df = df.append( pd.read_csv(f))
    
# print(df)
# df.to_csv("superimmo_20211008_13_annonces_cumul.csv")

      Unnamed: 0                                               link  \
0              0  https://www.superimmo.com/annonces/achat-maiso...   
1              1  https://www.superimmo.com/annonces/achat-maiso...   
2              2  https://www.superimmo.com/annonces/achat-appar...   
3              3  https://www.superimmo.com/annonces/achat-appar...   
4              4  https://www.superimmo.com/annonces/achat-maiso...   
...          ...                                                ...   
1652        1652  https://www.superimmo.com/annonces/achat-maiso...   
1653        1653  https://www.superimmo.com/annonces/achat-maiso...   
1654        1654  https://www.superimmo.com/annonces/achat-maiso...   
1655        1655  https://www.superimmo.com/annonces/achat-appar...   
1656        1656  https://www.superimmo.com/annonces/achat-appar...   

                                                    ref  \
0     Publiée le 24/08/2021 - maj le 24/09/2021 - Ré...   
1     Publiée le 01/09/2021 -

In [10]:
# df = pd.read_csv('superimmo_20211008_13_annonces_cumul.csv')
df = synthese
df

,link,ref,ref_si,location,agence,description,details,titre
0,https://www.superimmo.com/annonces/achat-appar...,Publiée le 28/05/2021 - maj le 08/11/2021 - Ré...,ANVXLWRA1,Paris (75),LE MAS SAINT COME (125 avis) Publiée le 28/05/...,NOUVEAUX APPARTEMENTS DISPONIBLES\nVenez décou...,Prix de vente : 674 000 €|Prix au m² : 12 255 ...,Vente appartement 55 m² - 2 pièces - 1 chambre...
1,https://www.superimmo.com/annonces/achat-appar...,Publiée le 18/11/2021 - maj le 01/12/2021 - Ré...,ANVXMXWUB,Paris 18ème (75018),Lifestone Grand Paris Publiée le 18/11/2021 - ...,EXCLUSIVITE / PARIS 18eme - Avenue Junot - Pla...,Prix de vente : 319 000 €|Prix au m² : 13 870 ...,Vente appartement 23 m² - 1 pièce\nParis 18ème...
2,https://www.superimmo.com/annonces/achat-appar...,Publiée le 05/10/2021 - maj le 16/12/2021 - Ré...,ANVXMNF3T,Paris 17ème (75017),Pierre de Taille Immobilier Publiée le 05/10/2...,"17è quartier Ternes, dans ce bel immeuble pier...",Prix de vente : 2 190 000 €|Prix au m² : 12 51...,Vente appartement 175 m² - 6 pièces - 4 chambr...
3,https://www.superimmo.com/annonces/achat-appar...,Publiée le 03/09/2021 - maj le 16/12/2021 - Ré...,ANVXMGR9H,Paris 3ème (75003),Paris Luxury Homes Publiée le 03/09/2021 - maj...,À VENDRE PARIS 3ÈME - BEAUBOURG - COUP DE COEU...,Prix de vente : 749 000 €|Prix au m² : 14 980 ...,Vente appartement 50 m² - 2 pièces - 1 chambre...
4,https://www.superimmo.com/annonces/achat-appar...,Publiée le 03/06/2021 - maj le 16/12/2021 - Ré...,ANVXLYA7N,Paris 17ème (75017),A la lucarne de l'immobilier Publiée le 03/06/...,"EN NOUVEAUTE Avenue de Wagram, cliquez sur VIS...",Prix de vente : 750 000 €|Prix au m² : 12 931 ...,Vente appartement 58 m² - 2 pièces - 1 chambre...
5,https://www.superimmo.com/annonces/achat-appar...,Publiée le 29/09/2021 - maj le 30/11/2021 - Ré...,ANVXMM8Q3,Paris 20ème (75020),iad FRANCE - Cécile DEMEURE Publiée le 29/09/2...,iad France - Cécile DEMEURE Afficher le numéro...,Prix de vente : 449 000 € |Prix au m² : 8 315 ...,Vente appartement 54 m² - 3 pièces - 2 chambre...
6,https://www.superimmo.com/annonces/achat-appar...,Publiée le 26/06/2021 - maj le 16/12/2021 - Ré...,ANVXM3UV6,Paris 20ème (75020),PIERRE IMMOBILIER Publiée le 26/06/2021 - maj ...,"PARIS 20EME // Proximité Métro Saint Fargeau, ...",Prix de vente : 577 200 €|Prix au m² : 10 126 ...,Vente appartement 57 m² - 2 pièces - 1 chambre...
7,https://www.superimmo.com/annonces/achat-appar...,Publiée le 17/02/2021 - maj le 16/12/2021 - Ré...,ANVXLAR02,Paris 13ème (75013),Immobiliere Initiale Paris Lowendal Publiée le...,13ème limite 14ème ; Mo GLACIÈRE. vendu par IM...,Prix de vente : 747 000 € |Prix au m² : 8 788 ...,Vente appartement 85 m² - 5 pièces - 3 chambre...


In [11]:
# formatage des données
# transformation des données en données interpretable par un modèle.

# beacoup des informations sont dans la colonne "details" ou on a mis toutes les informations du biens

# récupération des informations de "détails"
print(df.details.apply(lambda x: str(x).split('|')))

# print(df.details)

# recensement des critères de details : 
# information que l'on veut recueillir sur une annonce

# print(df.iloc[0].titre)
l=[]
for items in df.details.apply(lambda x: str(x).split('|')).to_list():
    for item in items:
        l.append(item)
print(l.sort())
print(l) 

# print(df.iloc[0].details.split('|'))

0    [Prix de vente : 674 000 €, Prix au m² : 12 25...
1    [Prix de vente : 319 000 €, Prix au m² : 13 87...
2    [Prix de vente : 2 190 000 €, Prix au m² : 12 ...
3    [Prix de vente : 749 000 €, Prix au m² : 14 98...
4    [Prix de vente : 750 000 €, Prix au m² : 12 93...
5    [Prix de vente : 449 000 € , Prix au m² : 8 31...
6    [Prix de vente : 577 200 €, Prix au m² : 10 12...
7    [Prix de vente : 747 000 € , Prix au m² : 8 78...
Name: details, dtype: object
None
['', '', '', '', '', '', '', '', '', ' ', ' ', '1 Parking', '1 balcon (6 m²)', "1 salle d'eau", "1 salle d'eau - 1 salle de bain", '1 salle de bain', '1 salle de bain', '1 salle de bain', '1 wc', '1 wc', '140 kWh/m².an', '140 kWh/m².an 40 kg CO₂/m².an D', '151 à 230\nD', '151 à 230\nD', '175 m² - 6 pièces - 4 chambres', '1er étage', '2 wc', '21 à 35\nD', '23 m² - 1 pièce', '231 à 330\nE', '36 à 55\nE', '40 kg CO₂/m².an', '50 m² - 2 pièces - 1 chambre', '54 m² - 3 pièces - 2 chambres', '55 m² - 2 pièces - 1 chambre', '56 

In [14]:
#recupération d'informations issue de details
# print(df.details)
import re

# données initiales
initial_datas = ["link","ref","agence","description","details","titre"]

parseDatas = {}
for initial_data in initial_datas :
    parseDatas[initial_data] = []

# données complémentaires deduites
match_keys = {
    'Prix de vente :':'prix',
    'Taxe foncière :':'taxe_fonciere',
    'Charges prévisionnelles :':'charges',
    'Orientation :':'orientation',
    'Classe énergie :':'classe_energie',
    'Année de construction :':'build_date',
    'DPE :':'DPE',
}
for ke in match_keys.values():
    parseDatas[ke]=[]

# ligne : "26 m² - 2 pièces - 1 chambre" attire particulièrement notre attention
# on effectuera un traitement particulier pour récupérer ses données là
parseDatas['surface']=[]
parseDatas['piece']=[]
parseDatas['chambre']=[]
parseDatas['location']=[]
parseDatas['type']=[]
parseDatas['terrain']=[]

#print(df.shape)
#print(df)
#print(df.info())
for index, row in df.iterrows():
    # reattribution des données initales
    for initial_data in initial_datas :
        parseDatas[initial_data].append(row[initial_data])
    
    # recensement des données complémentaires
    for k,v in match_keys.items():
        setValue = False
        alreadySet = [] # evite les doublons notamment pour dpe
        # un peu lourd de boucler sur les éléments de "détails" 
        # mais ça marche 
        # j'ai peu galéré sur cette partie 
        # notamment pour respecter le format et eviter les confusions de lecteur
        for d in str(row["details"]).split('|'): 
            if k in d:
                if v not in alreadySet : 
                    datatype = d.split(':')
                    if datatype[1] : 
                        alreadySet.append(v)
                        parseDatas[v].append(datatype[1])
                        setValue = True
                        
        if not setValue:
            parseDatas[v].append('')
    
    
    # même chose un peu lourd de boucler sur toutes les éléments de détails

    if "m² -" in row["titre"] : # sinon matche avec des mots comme sud-ouest, ...
        # en espérant qu'ils aient tous le mêmes format
        title_content = row["titre"].split("\n")
        # get location
        try :
            parseDatas['location'].append(title_content[1])
        except:
            parseDatas['location'].append('')
        
        # get mesures
        mesures = title_content[0].split("-")
        # ['110 m² ', ' 5 pièces ', ' 3 chambres']

        # j'utilise ici des try pour eviter les erreurs sur les éléments non définis
        # notamment dans ce cas : 10 m² - 1 pièce (où chambre n'est pas défini)
        try :
            result = re.search(r"Vente (\D*) ([\d,]*) m²", mesures[0])
            parseDatas['surface'].append(result.group(2))
            parseDatas['type'].append(result.group(1))
        except:
            parseDatas['surface'].append('')
            parseDatas['type'].append('')

        try :
            parseDatas['piece'].append(mesures[1]) 
        except:
            parseDatas['piece'].append('')

        try : 
            parseDatas['chambre'].append(mesures[2]) 
        except:
            parseDatas['chambre'].append('')
            
        try : 
            parseDatas['terrain'].append(mesures[3]) 
        except:
            parseDatas['terrain'].append('')
    else :
        parseDatas['surface'].append('')
        parseDatas['piece'].append('')
        parseDatas['chambre'].append('')
        parseDatas['location'].append('')
        parseDatas['type'].append('')
        parseDatas['terrain'].append('')
        
                
                        
# print(parseDatas)
# for k,v in parseDatas.items():
#     print(k,len(v))
# transforamtion des données en dataframe
synthese = pd.DataFrame(parseDatas)
print(synthese)

# et sauvegarde des resultats : 
# synthese.to_csv("superimmo_20211008_13_cumul_step1.csv")


# commentaire : 
# bien sur on peut chercher d'avantage d'éléments significatifs
# certaines annonces mentionne l'année de construction
# d'autres si le bien comprend un cave, un balcon ou un parking
# ou encore certaines précise l'étage ...


                                                link  \
0  https://www.superimmo.com/annonces/achat-appar...   
1  https://www.superimmo.com/annonces/achat-appar...   
2  https://www.superimmo.com/annonces/achat-appar...   
3  https://www.superimmo.com/annonces/achat-appar...   
4  https://www.superimmo.com/annonces/achat-appar...   
5  https://www.superimmo.com/annonces/achat-appar...   
6  https://www.superimmo.com/annonces/achat-appar...   
7  https://www.superimmo.com/annonces/achat-appar...   

                                                 ref  \
0  Publiée le 28/05/2021 - maj le 08/11/2021 - Ré...   
1  Publiée le 18/11/2021 - maj le 01/12/2021 - Ré...   
2  Publiée le 05/10/2021 - maj le 16/12/2021 - Ré...   
3  Publiée le 03/09/2021 - maj le 16/12/2021 - Ré...   
4  Publiée le 03/06/2021 - maj le 16/12/2021 - Ré...   
5  Publiée le 29/09/2021 - maj le 30/11/2021 - Ré...   
6  Publiée le 26/06/2021 - maj le 16/12/2021 - Ré...   
7  Publiée le 17/02/2021 - maj le 16/12/2021 - 

In [12]:
# formatage : certains prix sont des nombres à virgules (voir streamlit)
# df=pd.read_csv('superimmo_20211008_13_cumul_step1.csv')
# df.prix.apply()

#df.to_csv("superimmo_20211008_13_cumul_step1.csv")

0         410 000 €
1         350 000 €
2         178 500 €
3          96 000 €
4        189 000 € 
           ...     
9351      295 000 €
9352      412 000 €
9353      430 000 €
9354      290 000 €
9355      190 000 €
Name: prix, Length: 9356, dtype: object

In [ ]:
# conclusion : 
# on obtient un dataframe qui ressemble à peu près à ça

"""

link  \
0  https://www.superimmo.com/annonces/achat-appar...   
1  https://www.superimmo.com/annonces/achat-appar...   
2  https://www.superimmo.com/annonces/achat-appar...   
3  https://www.superimmo.com/annonces/achat-appar...   
4  https://www.superimmo.com/annonces/achat-appar...   

                                                 ref  \
0  Publiée le 24/06/2021 - maj le 16/09/2021 - Ré...   
1  Publiée le 23/07/2021 - maj le 28/09/2021 - Ré...   
2  Publiée le 12/06/2021 - maj le 28/09/2021 - Ré...   
3  Publiée le 15/06/2021 - maj le 05/09/2021 - Ré...   
4  Publiée le 09/07/2021 - maj le 26/09/2021 - Ré...   

                                              agence  \
0  ESPACES ATYPIQUES PARIS RIVE DROITE (316 avis)...   
1  Immobiliere Initiale Paris Lowendal Publiée le...   
2  Pierre de Taille Immobilier Publiée le 12/06/2...   
3  I@D France - Lea MIGUET Publiée le 15/06/2021 ...   
4  C.O.I SARL (Corinne Orbach Immobilier) Publiée...   

                                         description  \
0  Au calme d'une cour pavée et arborée, situé en...   
1  Appartement d'angle, bel ancien de caractère a...   
2  Nouvelle annonce immobilière pour un apparteme...   
3  iad France - Lea MIGUET Afficher le numéro) vo...   
4  Emplacement exceptionnel et coup de coeur abso...   

                                             details           prix  \
0  Prix de vente : 1 210 000 €|Prix au m² : 11 00...    1 210 000 €   
1  Prix de vente : 995 000 € |Prix au m² : 11 845...     995 000 €    
2  Prix de vente : 1 398 000 € |Prix au m² : 12 9...   1 398 000 €    
3  Prix de vente : 575 000 €|Prix au m² : 8 984 €...      575 000 €   
4  Prix de vente : 298 000 €|Prix au m² : 28 904 ...      298 000 €   

  taxe_fonciere        charges orientation                  classe_energie  \
0       1 087 €   2 302 € / an                 6 kg eq.CO₂/m².an 6 à 10\nB   
1                 3 984 € / an   sud-ouest   30 kg eq.CO₂/m².an 21 à 35\nD   
2                                                                            
3                 3 272 € / an               24 kg eq.CO₂/m².an 21 à 35\nD   
4                   516 € / an                                               

                             DPE    surface       piece      chambre  
0   197 kWhEP/m².an 151 à 230\nD    110 m²    5 pièces    3 chambres  
1   186 kWhEP/m².an 151 à 230\nD     84 m²    4 pièces    2 chambres  
2                                   108 m²    5 pièces    3 chambres  
3    106 kWhEP/m².an 91 à 150\nC     64 m²    3 pièces    2 chambres  
4                                 10,31 m²      1 pièce               

"""

In [ ]:
# formatage des données
# pour pouvoir utilisé les données de ce dataframe il faut formater des notamment le prix , la surface
# on peut/doit aussi 
# reduire le nombres pieces et de chambre à un entier 
# exprimer le DPE seulement par la lettre,  etc ...

In [ ]:
# formatage du prix de vente
# print(df.prix)

# print(df.prix.apply(lambda x : str(x).replace("€","").replace(' ','')).value_counts())
# visiblement on a des annonces sans prix : 
# nan        55   
# 630000     22
# 499000     17
# 750000     17
# 650000     14
#            ..
# 999500      1
# 109000      1
# 1500000     1
# 626320      1
# 628800      1
# Name: prix, Length: 737, dtype: int64

df.prix = df.prix.apply(lambda x : str(x).replace("€","").replace(' ','')
# ?? c'est peut-être les annonces qui demandais de saisir son mail pour accéder à l'annonce....
# todo : chercher une explication 
# on les négligera pour le moment
df = df[-(df['prix']=='nan')]
df['prix'] = df['prix'].astype(int)

df.info()
                        
"""
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1606 entries, 0 to 1660
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      1606 non-null   int64 
 1   link            1606 non-null   object
 2   ref             1602 non-null   object
 3   agence          1606 non-null   object
 4   description     1596 non-null   object
 5   details         1606 non-null   object
 6   prix            1606 non-null   int64 
 7   taxe_fonciere   366 non-null    object
 8   charges         1176 non-null   object
 9   orientation     524 non-null    object
 10  classe_energie  720 non-null    object
 11  DPE             751 non-null    object
 12  surface         1493 non-null   object
 13  piece           1493 non-null   object
 14  chambre         1208 non-null   object
dtypes: int64(2), object(13)
memory usage: 200.8+ KB
"""

In [ ]:
# formatage de la surface : 
# même travail sur la surface
print(df.surface)
df.surface = df.surface.apply(lambda x : str(x).replace("m²","").replace(' ','').replace(',','.'))
# même questionnement sur des biens sans surface
df = df[-(df['surface']=='nan')]
# print(df)

df.surface = df['surface'].astype(float)

df.info()

"""
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1493 entries, 0 to 1660
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      1493 non-null   int64  
 1   link            1493 non-null   object 
 2   ref             1489 non-null   object 
 3   agence          1493 non-null   object 
 4   description     1483 non-null   object 
 5   details         1493 non-null   object 
 6   prix            1493 non-null   int64  
 7   taxe_fonciere   366 non-null    object 
 8   charges         1084 non-null   object 
 9   orientation     450 non-null    object 
 10  classe_energie  672 non-null    object 
 11  DPE             704 non-null    object 
 12  surface         1493 non-null   float64
 13  piece           1493 non-null   object 
 14  chambre         1208 non-null   object 
dtypes: float64(1), int64(2), object(12)
memory usage: 186.6+ KB
"""

In [ ]:
# conclusion
# il y a encore pas mal de pistes à envisager : 
# 
# Extraire davantage de données:
# le prix au metre carré
# le prix au mettre carré par arrondissement
# répartition des biens par agence: 
#      y a-t-il des agences qui surévalue ou sous évalue des biens?
#.     y a-t-il des agences qui ne traite des biens qu'à partir d'un certains montant?
#      (normalement les frais d'agences sont à la charge du vendeur)
# lire la description de l'annonce : 
#     trouver un moyen de lire les éléments importants de la description de l'annonce.
#     "travaux à prévoir", vue ... au autre éléments 
#     certaines mentionnent des éléments géographique proche metro ..., en face du parc ..., à deux pas de ...
# extraire de "details" d'autres variables significative. (année de constuction,cave, parking, etage,...)
# 
# 
# Analyse des données:
# gestion des outliers.
# chercher les corrélations entre les variables.
#     le prix des charges en fonction du prix
# 
# 
# Déduire davantage de données singnficatives (cherchant à expliquer le prix)
# trouver les variables qui pèse le plus dans le prix (shape, lime, coefficent de regression lasso)
# Entrainer des modèles des regressions 
# 
# 
# Evolution des prix : 
# avec les biens similaires vendus les années auparavant. cf dvf 2020,2019,...sur les mêmes arrondissements.
# certaines annonces précise une baisse de prix 
# en faisant cette même opération régulièrement on pourrait voir aussi 
#       les biens qui partent vite
#       les biens qui baissent leurs prix
#       ...
# (en espérant que le site qu'on utilise ne mette pas de protection sur leurs d'ici là ...)
# 
# 
# Remarques : 
# il faut sans doute prendre un peu de recul par rapport au données que l'on a 
#     utiliser peut-être d'autres sources de données
# je n'ai pas tenu compte de la qualité des photos des annonces
#